The dataset talks about daily views from different wikipedia articles. We can extract a lot of useful information form the page name like the language of the article and source as well as the access point.

This article tries to analyze the data to some extent.
Since it is a time series LSTM should work for predictions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dataset = pd.read_csv('../input/train_1.csv')

In [3]:
dataset.head()

In [4]:
len(dataset)

In [5]:
page = dataset["Page"]

In [6]:
page.head()

In [7]:
# Name of the article
name = []

# Project the article was search in
project = []

# The object used to access the article
access = []

# Agent
agent = []

In [8]:
# Spliting the information in the page
for i in range(len(dataset)):
    split_row = page[i].split('_')
    j = len(split_row)-1
    agent.append(split_row[j])
    j -= 1
    access.append(split_row[j])
    j -= 1
    project.append(split_row[j])
    j -= 1
    while(j>=0):
        name.append(split_row[j])
        j -= 1

In [9]:
unique_agents = set(agent)
print(unique_agents)

In [10]:
unique_access = set(access)
print(unique_access)

In [11]:
unique_projects = set(project)
print(unique_projects)

In [12]:
language = []
source = []

In [13]:
# Extracting the language and source from project
for i in range(len(dataset)):
    split_row = project[i].split('.')
    if split_row[1] == 'wikipedia':
        language.append(split_row[0])
        source.append(split_row[1])
    else :
        language.append("Media")
        source.append(split_row[1])

In [14]:
unique_languages = set(language)
print(unique_languages)

In [15]:
unique_sources = set(source)
print(unique_sources)

In [16]:
print(len(source))   
    

In [17]:
dataset["language"] = language
dataset["source"] = source
dataset["access"] = access
dataset["agent"] = agent

In [18]:
dataset = dataset.fillna(0.0)

In [19]:
dataset.head()

In [20]:
sns.set(style="whitegrid", color_codes=True)
def plotFrequency(key):
    plt.figure(figsize=(12,8))
    ax = sns.countplot(x=key, data=dataset,palette="GnBu_d")
    plt.ylabel('Frequency')
    plt.title('Frequency distribution')
    plt.show()

plotFrequency("language")
plotFrequency("source")
plotFrequency("access")
plotFrequency("agent")

In [21]:
days = [r for r in range(dataset.shape[1]-5)]

def plotDailyViews(table, element):
    fig = plt.figure(1,figsize=[12,12])
    plt.ylabel('Views')
    plt.xlabel('Days')
    plt.title('Comparison of '+element)
    label={#languages
            'na':'Media',
            'de':'German',
            'en':'English',
            'es':'Spanish',
            'fr':'French',
            'ja':'Japanese',
            'ru':'Russian',
            'zh':'Chinese',
           #source
            'mediawiki':'mediawiki',
            'wikimedia':'wikimedia',
            'wikipedia':'wikipedia',
           #access
            'all-access':'all-access',
            'desktop':'desktop',
            'mobile-web':'mobile-web',
           #agents
            'all-agent':'all-agent',
            'spider':'spider'
           }
    done = None
    for key in label:
        if key in table.columns:
            plt.plot(days,table[key],label = label[key] )
    plt.legend()
    plt.show()

In [22]:
# Language daily views
language_daily_sum = dataset.groupby('language').sum().reset_index()
language_daily_sum_transpose = language_daily_sum.T
language_daily_sum_transpose.head()

In [23]:
language_daily_sum_transpose.columns=["na","de","en","es","fr","ja","ru","zh"]
language_daily_sum_transpose = language_daily_sum_transpose[1:]
plotDailyViews(language_daily_sum_transpose,"Languages")

English tops the list of the languages being searched in wikipedia

In [24]:
# Source daily views
source_daily_sum = dataset.groupby('source').sum().reset_index()
source_daily_sum_transpose = source_daily_sum.T
source_daily_sum_transpose.head()

In [25]:
source_daily_sum_transpose.columns=["mediawiki","wikimedia","wikipedia"]
source_daily_sum_transpose = source_daily_sum_transpose[1:]
plotDailyViews(source_daily_sum_transpose, "Sources")

Wikipedia has most number of daily views which in one way signifies people generally searches for text and comparitively a very few number of views are for medias

In [26]:
# Access daily views
access_daily_sum = dataset.groupby('access').sum().reset_index()
access_daily_sum_transpose = access_daily_sum.T
access_daily_sum_transpose.head()

In [27]:
access_daily_sum_transpose.columns=["all-access","desktop","mobile-web"]
access_daily_sum_transpose = access_daily_sum_transpose[1:]
plotDailyViews(access_daily_sum_transpose, "Access")

In [28]:
# Agent daily views
agent_daily_sum = dataset.groupby('agent').sum().reset_index()
agnet_daily_sum_transpose = agent_daily_sum.T
agnet_daily_sum_transpose.head()

In [29]:
agnet_daily_sum_transpose.columns=["all-agent","spider"]
agnet_daily_sum_transpose = agnet_daily_sum_transpose[1:]
plotDailyViews(agnet_daily_sum_transpose, "Agent")